In [1]:
import sys
sys.path.append("../../")
from STK_Sim import *

Filename = 'AERO_402_Further_Assessment'

stk_object = STK_Simulation(False,Filename)

In [20]:
# Loading targets into stk from file.
stk_object.set_sim_time(days=1)
stk_object.Target_Loader("../../Input_Files/Targets_15.txt")
stk_object.Satellite_Loader("../../Input_Files/Satellites_File.txt")

In [ ]:
stk_object.Compute_AzEl()

percentages = [np.count_nonzero(stk_object.target_bins[idx])/324*100 for idx in range(len(stk_object.targets))]
times = [np.max(stk_object.target_times[idx])/86400 for idx in range(len(stk_object.targets))]
print(f"Percentage = {np.average(percentages)}")
print(f"Time = {np.average(times)}")

In [ ]:
self = stk_object
enable_print = True

stk_object.set_sim_time(days=1)

self.Reset_Target_Bins()
with alive_bar(len(self.targets)*len(self.satellites),force_tty=True,bar='classic',title='- Computing_AzEl',length=10,disable=not(enable_print)) as bar:
    for sat in self.satellites:
        for tar_num,tar in enumerate(self.targets):
            access = tar.GetAccessToObject(sat)
            access.ComputeAccess()
            Intervals1 = access.DataProviders.GetItemByName('AER Data').Group.Item(0).ExecElements(self.root.CurrentScenario.StartTime,
                                                                                        self.root.CurrentScenario.StopTime,
                                                                                        self.dt,['Time','Azimuth','Elevation']).Intervals
            Intervals2 = access.DataProviders.GetItemByName('Sat Angles Data').ExecElements(self.root.CurrentScenario.StartTime,
                                                                                        self.root.CurrentScenario.StopTime,
                                                                                        self.dt,["Cross Track","Along Track"]).Intervals                                                                                                    
            for Int1,Int2 in zip(Intervals1,Intervals2):
                print("HERE")
                times_az = Int1.MultipleThresholdCrossings("Azimuth",az_range)
                times_el = Int1.MultipleThresholdCrossings("Elevation",el_range)
                for i,t_az in enumerate(times_az[1:]):
                    for j,t_el in enumerate(times_el[1:]):
                        if len(t_az)>0 and len(t_el)>0:
                            for t in t_el:
                                if (any(t[0] <= value <= t[1] for value in t_az[0]) or 
                                    any(t_az[0][0] <= value <= t_az[0][1] for value in t)):
                                    Int2.ThresholdCrossings("Time",t[0])
            bar()